<a href="https://colab.research.google.com/github/zjay1995/EE209_Computational_Robotics/blob/master/Self_balancing_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import FaBo9Axis_MPU9250
import time
import sys
import math
import RPi.GPIO as GPIO


P=2.5
I=2.0
D=1.0
runner = 500

class PIDController:
    def __init__(self, P, I, D):
        self.KP = P
        self.KI = I
        self.KD = D
        self.target = 0

        self.lastError = 0
        self.integrator = 0

    def setTarget(self, newTarget):
        self.target = newTarget
        self.integrator = 0

    def step(self, currentValue):
        error = currentValue - self.target

        output = (self.KP * error + self.KI * self.integrator + self.KD * (error - self.lastError))

        self.lastError = error
        self.integrator += error

        return output
  
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BOARD)
servoPin=11
GPIO.setup(servoPin, GPIO.OUT)
pwm=GPIO.PWM(servoPin,50)
pwm.start(0)

def move(velocity):
    DC = 1./18.*(velocity+90.0) + 2
    pwm.ChangeDutyCycle(DC)

def equi():
    vstop = 1./18.*(90) + 2
    pwm.ChangeDutyCycle(vstop)

#K and K1 --> Constants used with the complementary filter
K = 0.98
K1 = 1 - K

time_diff = 0.01
ITerm = 0

mpu9250 = FaBo9Axis_MPU9250.MPU9250()

#Calling the MPU9250 data 
accel_data = mpu9250.readAccel()
gyro_data = mpu9250.readGyro()

aTempX = accel_data['x']
aTempY = accel_data['y']
aTempZ = accel_data['z']

gTempX = gyro_data['x']
gTempY = gyro_data['y']
gTempZ = gyro_data['z']
print(aTempX)
#some math 
def distance(a, b):
    return math.sqrt((a*a) + (b*b))

def y_rotation(x, y, z):
    radians = math.atan2(x, distance(y, z))
    return -math.degrees(radians)

def x_rotation(x, y, z):
    radians = math.atan2(y, distance(x, z))
    return math.degrees(radians)
  
last_x = x_rotation(aTempX, aTempY, aTempZ)
last_y = y_rotation(aTempX, aTempY, aTempZ)

gyro_offset_x = gTempX
gyro_offset_y = gTempY

gyro_total_x = (last_x) - gyro_offset_x
gyro_total_y = (last_y) - gyro_offset_y
i = 0
angledata =[]
for i in range(runner):
    accel = mpu9 250.readAccel()
    accelX = accel['x']
    accelY = accel['y']
    accelZ = accel['z']

    gyro = mpu9250.readGyro()
    gyroX = gyro['x']
    gyroY = gyro['y']
    gyroZ = gyro['z']

    gyroX -= gyro_offset_x
    gyroY -= gyro_offset_y
        
    gyro_x_delta = (gyroX * time_diff)
    gyro_y_delta = (gyroY * time_diff)

    gyro_total_x += gyro_x_delta
    gyro_total_y += gyro_y_delta

    rotation_x = x_rotation(accelX, accelY, accelZ)
    rotation_y = y_rotation(accelX, accelY, accelZ)
    #Complementary Filter
    last_x = K * (last_x + gyro_x_delta) + (K1 * rotation_x)

    #setting the PID values. Here you can change the P, I and D values according to yiur needs
    #print(int(last_x))
    PID = PIDController(P, I, D)
    PIDx = PID.step(last_x)

    #if the PIDx data is lower than 0.0 than move appropriately backward
    if PIDx != 0.0:
        move(float(PIDx))
        #StepperFor(-PIDx)
    #if the PIDx data is higher than 0.0 than move appropriately forward
    else:
        equi()
    angledata.append(last_x)                
    time.sleep(0.01)
    i+=1
      
print(angledata)
pwm.stop()
GPIO.cleanup()